# Team Project - Elections - Silver Annotation

## Importing all the libraries

In [4]:
import pandas as pd
import ollama

import nltk
from nltk.corpus import stopwords
import string
import re
import emoji

In [3]:
!pip install emoji
!pip install ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 5.4 MB/s eta 0:00:0000:0100:01
  Using cached ollama-0.1.7-py3-none-any.whl (9.4 kB)
  Using cached httpx-0.25.2-py3-none-any.whl (74 kB)
  Using cached httpcore-1.0.4-py3-none-any.whl (77 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


### Reading the JSON file

In [5]:
df = pd.read_json("/Users/niveditamenon/Downloads/05_Nivi.json", lines=True)

## Pre-processing

In [6]:
# Remove _id column

df = df.drop(columns=["_id"])

In [7]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/niveditamenon/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Helper Functions

In [8]:
def remove_stopwords_and_punctuation(tweet_column):
    stop_words = set(stopwords.words("english"))
    cleaned_tweets = []

    for tweet in tweet_column:
        # Remove punctuation
        tweet = tweet.translate(str.maketrans("", "", string.punctuation))

        # Remove stop words
        words = tweet.split()
        filtered_words = [
            word.lower() for word in words if word.lower() not in stop_words
        ]
        cleaned_tweets.append(" ".join(filtered_words))

    return cleaned_tweets

In [21]:
def remove_mentions(tweet_column):
    # Use regular expression to remove mentions
    pattern = re.compile(r"@[a-zA-Z0-9_]+")
    cleaned_tweets = [re.sub(pattern, "@@mention", tweet) for tweet in tweet_column]
    return cleaned_tweets

In [10]:
def tokenize_tweets(tweet_column):
    # Split and strip each tweet into tokens
    tokenized_tweets = [tweet.split() for tweet in tweet_column]
    return tokenized_tweets

In [11]:
def convert_emoji_to_emotion(tweet_column):
    # Convert emojis to corresponding emotions
    converted_tweets = [emoji.demojize(tweet) for tweet in tweet_column]
    return converted_tweets

In [ ]:
# Remove links
def remove_links(tweet_column):
    # Use regular expression to remove links
    pattern = re.compile(r"http[s]?://\S+")
    cleaned_tweets = [re.sub(pattern, "", tweet) for tweet in tweet_column]
    return cleaned_tweets

In [ ]:
# Tokenise the tweets
def tokenize_tweets(tweet_column):
    # Split and strip each tweet into tokens
    tokenized_tweets = [tweet.split() for tweet in tweet_column]
    return tokenized_tweets

In [ ]:
# Remove emojis
def remove_emojis(tweet_column):
    # Use regular expression to remove emojis
    pattern = re.compile(r"(:\w+:)")
    cleaned_tweets = [re.sub(pattern, "", tweet) for tweet in tweet_column]
    return cleaned_tweets

### Applying all the functions

In [22]:
# Apply the text processing functions to the 'tweets' column

df["anonymized_tweets"] = remove_mentions(df["text"])
df["cleaned_tweets"] = remove_stopwords_and_punctuation(df["anonymized_tweets"])
df["cleaned_tweets"] = convert_emoji_to_emotion(df["cleaned_tweets"])
df["tokenized_tweets"] = tokenize_tweets(df["cleaned_tweets"])

In [20]:
# Display the DataFrame

print(df["text"][1], df["tokenized_tweets"][1])

@MattHancock you fucking scrotbag ['mention', 'fucking', 'scrotbag']


### OLLAMA Annotation

### Designing prompt

In [15]:
def promt_generator(df, i):
    prompt = (
        """
        Task:** Assign one or more labels to the following text. Each label corresponds to a specific category. Please read the text and select all the relevant labels.
        Labels:**
        1. Anti-EU-related narratives
        2. Anti-Elites
        3. Distrust in democratic system-related narratives
        4. Distrust in institutions-related narratives
        5. Ethnic-related narratives
        6. Gender-related narratives
        7. Geopolitics
        8. Migration-related narratives
        9. Political hate-related narratives
        10.Religious-related narratives

        **Text to be categorized:**
        """
        + str(df["tokenized_tweets"][i])
        + """

        **Instructions:**
        1. Read the provided text carefully.
        2. Determine all the relevant categories that apply to the text.
        3. Choose the corresponding label numbers (1-10) that best represent the content.
        4. If the text doesn't fit any of the labels, select the one that is the closest match.

        OUTPUT: Please provide ONLY the label numbers separated by commas WITHOUT additional explanation. For example, if you think the text fits Label_1 and Label_3, you should enter "1, 3".
        """
    )
    return prompt

### Tweet ID-wise Ollama Anatotion

In [ ]:
for i in range(0, 2):
    prompt = promt_generator(df, i)
    print(i)
    print(ollama.generate(model="llama2", prompt=prompt)["response"])